# Libraries & Merge Datasets

## Libraries

In [1]:
import pandas as pd

## Load Dataset

In [2]:
precovid = pd.read_csv('../rawdata/escooters-precovid.csv', index_col = 'TripID')

In [3]:
precovid.isnull().sum()

StartDate         0
StartTime         0
EndDate           4
EndTime           4
TripDuration      0
TripDistance      0
StartLatitude     0
StartLongitude    0
EndLatitude       0
EndLongitude      0
DayOfWeek         0
HourNum           0
dtype: int64

In [4]:
postcovid = pd.read_csv('../rawdata/escooters-postcovid.csv', index_col = 'TripID')

In [5]:
postcovid.isnull().sum()

StartDate         0
StartTime         0
EndDate           0
EndTime           0
TripDuration      0
TripDistance      0
StartLatitude     0
StartLongitude    0
EndLatitude       0
EndLongitude      0
DayOfWeek         0
HourNum           0
dtype: int64

## Merge Dataset and drop NaN

In [6]:
df = precovid.append(postcovid)

In [7]:
df.isnull().sum()

StartDate         0
StartTime         0
EndDate           4
EndTime           4
TripDuration      0
TripDistance      0
StartLatitude     0
StartLongitude    0
EndLatitude       0
EndLongitude      0
DayOfWeek         0
HourNum           0
dtype: int64

In [8]:
df.dropna(inplace = True)

In [9]:
df.dtypes

StartDate          object
StartTime          object
EndDate            object
EndTime            object
TripDuration      float64
TripDistance      float64
StartLatitude     float64
StartLongitude    float64
EndLatitude       float64
EndLongitude      float64
DayOfWeek           int64
HourNum             int64
dtype: object

# Replace Invalid Values

We want to fix those lines where the hour is indicated as 24 instead of 00, and fix the day of the week accordingly.

## HourNum

In [10]:
df['HourNum'].replace(24, 0, inplace = True)
df['HourNum'].loc[df['HourNum'] == 0]

TripID
0041c164-abfd-a85b-ac9d-60bafc43    0
0041d45b-b62c-d892-a67c-ecbfa07e    0
006b741a-274b-7975-116b-6a9f988a    0
00745ffa-ee7d-8527-2ebf-1eb18441    0
007ba6d2-48e4-366d-8675-2bf0f834    0
                                   ..
ff5c22f5-5fe3-7b45-2a8d-40684a2c    0
ff5d15d9-d2b6-990a-079c-4e72af90    0
ffcdc2f5-95d0-55eb-d301-ce3b82e7    0
ffe4cade-ff8b-fe8f-ef6a-39a21d05    0
ffee9c70-c1b6-1ed9-2779-7e14795f    0
Name: HourNum, Length: 2440, dtype: int64

## StartTime


In [11]:
invalid = df.loc[ df['StartTime'].str.contains('24:') ].index

In [12]:
df['StartTime'].loc[invalid] = df['StartTime'].loc[invalid].str.replace('24:', '00:')

/Users/luca/.pyenv/versions/miniconda3-latest/envs/mobility/lib/python3.8/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [13]:
df['Start'] = df['StartDate'] + ' ' + df['StartTime']

In [14]:
df['Start'] = pd.to_datetime(df['Start'])

In [15]:
df['Start'].loc[invalid] = df['Start'].loc[invalid] + pd.Timedelta(days=1)
df['Start'].loc[invalid]

TripID
007ba6d2-48e4-366d-8675-2bf0f834   2019-06-02
01688a3a-ecb1-742e-1ddf-ed31f3f8   2019-07-13
018ff9a4-ef24-bda3-ac05-820157ea   2019-07-27
01c27413-ad2e-7fe2-b354-b7e4a9f1   2019-07-27
01db3512-b974-aa8b-887a-2d4ae44d   2018-12-21
                                      ...    
fe3cc91c-d393-a343-92a2-288903f1   2019-09-22
fe3d6651-d91d-f94c-9de5-8dd2ff3f   2019-06-20
fe8cb696-048e-03f3-16f5-1d62826c   2019-07-29
ffcdc2f5-95d0-55eb-d301-ce3b82e7   2019-08-06
ffee9c70-c1b6-1ed9-2779-7e14795f   2019-04-30
Name: Start, Length: 799, dtype: datetime64[ns]

## EndTime

In [16]:
other = df.loc[ df['EndTime'].str.contains('24:') ].index
other

Index(['000e3f77-912c-84f8-b8b2-3c62c74c', '005a479a-ed28-98cb-5661-8018fd77',
       '007ba6d2-48e4-366d-8675-2bf0f834', '008e4642-04c9-d5d2-467c-403248e0',
       '0139f0e8-2e7e-cbee-bdae-bf47fc1c', '016fb00d-c238-68bd-4f0c-d1f7cfd7',
       '018ff9a4-ef24-bda3-ac05-820157ea', '0287129b-b488-977e-c97b-8ec3a129',
       '029b9b3a-3c16-95e5-b30a-953ae9bd', '02fa0d59-2633-020f-eae8-4999a5a6',
       ...
       'fa803b01-f24f-415d-e2a0-4355bccc', 'faad9c0c-65fb-ad2a-f532-8589944c',
       'fb7b0217-6eb6-c655-87ee-ce0806de', 'fbee6f39-915e-09b1-4a7b-c6bf08ab',
       'fda32fe6-7e12-0ea1-d52e-78923779', 'fdbfdec4-40aa-64f3-10cb-74413b03',
       'fddd0477-47cc-5435-774f-d40dcc56', 'fdf5ef2f-7f29-c625-7c52-e003f083',
       'fe3d6651-d91d-f94c-9de5-8dd2ff3f', 'ff9dfdbe-cb13-e3cd-53cb-bf3fa809'],
      dtype='object', name='TripID', length=884)

In [17]:
df['EndTime'].loc[other] = df['EndTime'].loc[other].str.replace('24:', '00:')

In [18]:
df['End'] = df['EndDate'] + ' ' + df['EndTime']
df['End'] = pd.to_datetime(df['End'])

In [19]:
df['End'].loc[other]

TripID
000e3f77-912c-84f8-b8b2-3c62c74c   2019-06-14
005a479a-ed28-98cb-5661-8018fd77   2019-07-13
007ba6d2-48e4-366d-8675-2bf0f834   2019-06-01
008e4642-04c9-d5d2-467c-403248e0   2019-05-13
0139f0e8-2e7e-cbee-bdae-bf47fc1c   2019-07-08
                                      ...    
fdbfdec4-40aa-64f3-10cb-74413b03   2019-08-17
fddd0477-47cc-5435-774f-d40dcc56   2019-05-02
fdf5ef2f-7f29-c625-7c52-e003f083   2019-09-21
fe3d6651-d91d-f94c-9de5-8dd2ff3f   2019-06-19
ff9dfdbe-cb13-e3cd-53cb-bf3fa809   2019-04-27
Name: End, Length: 884, dtype: datetime64[ns]

In [20]:
df['End'].loc[other] = df['End'].loc[other] + pd.Timedelta(days=1)
df['End'].loc[other]

TripID
000e3f77-912c-84f8-b8b2-3c62c74c   2019-06-15
005a479a-ed28-98cb-5661-8018fd77   2019-07-14
007ba6d2-48e4-366d-8675-2bf0f834   2019-06-02
008e4642-04c9-d5d2-467c-403248e0   2019-05-14
0139f0e8-2e7e-cbee-bdae-bf47fc1c   2019-07-09
                                      ...    
fdbfdec4-40aa-64f3-10cb-74413b03   2019-08-18
fddd0477-47cc-5435-774f-d40dcc56   2019-05-03
fdf5ef2f-7f29-c625-7c52-e003f083   2019-09-22
fe3d6651-d91d-f94c-9de5-8dd2ff3f   2019-06-20
ff9dfdbe-cb13-e3cd-53cb-bf3fa809   2019-04-28
Name: End, Length: 884, dtype: datetime64[ns]

# Final Dataset (no outliers removed)

## Fix col order

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 538951 entries, 0000045c-2677-3a7d-4b73-cad99a57 to 1a7e9820-615e-5918-9ae6-ec5256332b20
Data columns (total 14 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   StartDate       538951 non-null  object        
 1   StartTime       538951 non-null  object        
 2   EndDate         538951 non-null  object        
 3   EndTime         538951 non-null  object        
 4   TripDuration    538951 non-null  float64       
 5   TripDistance    538951 non-null  float64       
 6   StartLatitude   538951 non-null  float64       
 7   StartLongitude  538951 non-null  float64       
 8   EndLatitude     538951 non-null  float64       
 9   EndLongitude    538951 non-null  float64       
 10  DayOfWeek       538951 non-null  int64         
 11  HourNum         538951 non-null  int64         
 12  Start           538951 non-null  datetime64[ns]
 13  End             538951 non-null  

In [22]:
df.drop(axis = 1, columns = ['StartDate', 'StartTime', 'EndDate', 'EndTime'], inplace = True)

In [23]:
oldcol = [ col for col in df.columns if col not in ['Start', 'End'] ]
oldcol

['TripDuration',
 'TripDistance',
 'StartLatitude',
 'StartLongitude',
 'EndLatitude',
 'EndLongitude',
 'DayOfWeek',
 'HourNum']

In [24]:
newcol = ['Start', 'End']

for col in oldcol:
    newcol.append(col)

newcol

['Start',
 'End',
 'TripDuration',
 'TripDistance',
 'StartLatitude',
 'StartLongitude',
 'EndLatitude',
 'EndLongitude',
 'DayOfWeek',
 'HourNum']

In [25]:
df = df.reindex(columns=newcol)
df

,Start,End,TripDuration,TripDistance,StartLatitude,StartLongitude,EndLatitude,EndLongitude,DayOfWeek,HourNum
TripID,,,,,,,,,,
0000045c-2677-3a7d-4b73-cad99a57,2019-06-26 19:30:00,2019-06-26 19:30:00,3.0,0.000,38.253,-85.756,38.253,-85.755,4,19
0000487b-92e6-50d6-7569-42ed3818,2019-09-22 14:30:00,2019-09-22 14:30:00,5.0,0.000,38.203,-85.752,38.204,-85.751,1,14
00006088-2579-e0d0-6a30-a15bb878,2019-08-21 17:30:00,2019-08-21 17:30:00,6.0,0.330,38.259,-85.733,38.265,-85.739,4,17
00008c1a-899b-8596-970f-9f6bf495,2019-07-03 11:00:00,2019-07-03 11:15:00,6.0,0.640,38.217,-85.757,38.221,-85.763,4,11
00009301-3225-2aea-a84a-165a480a,2019-11-22 10:45:00,2019-11-22 11:00:00,7.0,0.599,38.215,-85.759,38.222,-85.764,6,10
...,...,...,...,...,...,...,...,...,...,...
b2762b7e-bf5b-5f76-9d8d-84cb7665c240,2020-07-08 07:15:00,2020-07-08 07:30:00,11.0,0.002,38.224,-85.760,38.224,-85.760,4,7
3a812dca-b083-5e88-a59c-fdf19f14f769,2020-07-08 08:00:00,2020-07-08 09:30:00,84.0,0.339,38.263,-85.731,38.261,-85.737,4,8
ac526223-0238-50f1-8591-5965e7114059,2020-07-08 08:00:00,2020-07-08 08:30:00,33.0,2.658,38.234,-85.763,38.234,-85.742,4,7


# Drop invalid values and outliers

Explore the dataset to sort out outliers and invalid data

In [26]:
import matplotlib.pyplot as plt

## Distance outlieres & negative distances

In [27]:
df.shape # size before: to visualise how many rows we take out each time

(538951, 10)

In [28]:
print('There are {} trips longer than 10 miles'.format(df.TripDistance.loc[(df.TripDistance > 10)].count()))
print('There are {} trips longer than 20 miles'.format(df.TripDistance.loc[(df.TripDistance > 20)].count()))

There are 1225 trips longer than 10 miles
There are 113 trips longer than 20 miles


In [29]:
# drop trips greater than 20 miles

df.drop(df.loc[df.TripDistance > 20].index, inplace = True)
len(df)

538838

In [30]:
# count negative distances

print('There are {} trips smaller than zero!'.format(df.TripDistance.loc[(df.TripDistance < 0)].count()))

There are 82 trips smaller than zero!


In [31]:
# remove those distances

df.drop(df.loc[df.TripDistance < 0].index, inplace = True)
len(df.index)

538756

## Null time and duration, plus same start and end coordinates

Let's look at which values are equal to zero for Duration and Distance

In [32]:
# trips with null distance! What to do with them?

null_distance = df.TripDistance == 0
null_trips = (df.TripDistance == 0) & (df.TripDuration == 0)
same_coordinates = (df.StartLatitude == df.EndLatitude) & (df.StartLongitude == df.EndLongitude)

print(
    'there are {} trips with null distance'.format(len(df.loc[null_distance].index)),
    'there are {} trips with null distance and duration'.format(len(df.loc[null_trips])),
    'there are {} trips with the same starting and end coordinates:'.format(len(df.loc[same_coordinates].index)),
    'be careful not to drop all of those!',
    sep = '\n'
)

# Pay attention: these are values for which start and end coordinates are the same
# These have to be kept because one might also start from the same zone and turn back to it

there are 85359 trips with null distance
there are 2949 trips with null distance and duration
there are 70283 trips with the same starting and end coordinates:
be careful not to drop all of those!


One might carefully drop all lines with null trips and same coordinates. But the data provided has fuzzy coordinates, while distances are reported by the providers of the service. This means that we can think of the distance as true distance, because it is not derived from the coordinates!

In [33]:
df.drop(df.loc[null_trips].index, inplace = True)
len(df.index) # to check how many obs are left

535807

## Summary of the informations in the final dataset

We dropped:

1. 113 trips longer than 20 miles (note that all trips == 100 are outliers filtered by the Louisville Municipality).
1. 82 trips with negative distance.
1. 2949 null trips, that is with zero length and distance.

We started from a dataset with 538951 rows and we end up with a dataset with 535807.

In [34]:
# convert miles into meters

df.TripDistance = df.TripDistance.apply(lambda x: round(x*1609))
df.TripDistance

TripID
0000045c-2677-3a7d-4b73-cad99a57           0
0000487b-92e6-50d6-7569-42ed3818           0
00006088-2579-e0d0-6a30-a15bb878         531
00008c1a-899b-8596-970f-9f6bf495        1030
00009301-3225-2aea-a84a-165a480a         964
                                        ... 
b2762b7e-bf5b-5f76-9d8d-84cb7665c240       3
3a812dca-b083-5e88-a59c-fdf19f14f769     545
ac526223-0238-50f1-8591-5965e7114059    4277
e0c07dd2-3257-5138-82ae-d520c9ef8e1f    1797
1a7e9820-615e-5918-9ae6-ec5256332b20    2523
Name: TripDistance, Length: 535807, dtype: int64

# Last Tweakings

## Add Week Of Year Column & COVID-19 Dummy

In [35]:
df.dtypes

Start             datetime64[ns]
End               datetime64[ns]
TripDuration             float64
TripDistance               int64
StartLatitude            float64
StartLongitude           float64
EndLatitude              float64
EndLongitude             float64
DayOfWeek                  int64
HourNum                    int64
dtype: object

In [36]:
df.TripDuration = df.TripDuration.astype('int64')

In [37]:
df['WeekOfYear'] = df['Start'].dt.isocalendar().week.astype('int64')

In [38]:
df['Covid'] = np.where(df['Start'] >= '2020-03-21 00:00:00', 1, 0)

In [39]:
df.dtypes

Start             datetime64[ns]
End               datetime64[ns]
TripDuration               int64
TripDistance               int64
StartLatitude            float64
StartLongitude           float64
EndLatitude              float64
EndLongitude             float64
DayOfWeek                  int64
HourNum                    int64
WeekOfYear                 int64
Covid                      int64
dtype: object

# Export Dataset


In [40]:
df.rename(columns = { 'TripDuration' : 'Duration', 'TripDistance' : 'Distance'}, inplace = True)

In [41]:
df.to_csv('../data/tidy-scooters.csv')